# Practical Human in the loop of machine learning

Konfuzio SDK allows you to connect your machine learning pipeline to Konfuzio app in a simple and easy way so that you can benefit from its functionalities.

Most machine learning models rely on data that has been prepared by humans. But the most powerful systems are set up such that they allow both sides to interact continuously. These systems allow humans to give direct feedback to a model.

In this tutorial we will show how you can use the Konfuzio SDK package to include an easy feedback workflow in your training pipeline.

We will show how to:
- use the results from a model as annotations in the Konfuzio app
- provide feedback by simply accept or reject those results
- get back the revised annotations

With these revised annotations you can retrain your model and repeat the loop.
By doing this you can reduce the effort in the labelling of the data and improve the performance of your model.


Let's start...

First, be sure you have **installed** and **initialized** the konfuzio_sdk package as shown in the readme of the [repository](https://github.com/konfuzio-ai/Python-SDK).

In [ ]:
!pip install konfuzio-sdk

In [ ]:
!konfuzio_sdk init

Note:  
If you want to switch to a different project, you have to restart the variables in colab (run cell bellow).


In [ ]:
!kill -9 -1

Import the necessary functions from the Konfuzio SDK and other python libraries.

In [ ]:
import os
import glob
from konfuzio_sdk.data import Project, Annotation, Document, Label
from konfuzio_sdk.api import upload_file_konfuzio_api
from konfuzio_sdk.utils import get_file_type
from konfuzio_sdk import SUPPORTED_FILE_TYPES

Here we will use a model from the library flair as an example. In the next cells we are installing it and importing the necessary functions.
You can skip this if you are using other model.

In [ ]:
!pip install flair

In [ ]:
from flair.models import SequenceTagger
from flair.data import Sentence
from flair.trainers import ModelTrainer
from flair.datasets import ColumnCorpus

Load your project.

In [ ]:
my_project = Project()

In [ ]:
# If in the meantime there was changes in the project in the Konfuzio App, be sure you local project is updated.
# Everytime that there are changes in the Konfuzio App, the local project must be updated to have those changes.
my_project.update()

Upload your documents using the following code.

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# filter the documents by the supported file types
files_to_check = [f for f in glob.glob('*') if os.path.isfile(f) and not f.endswith('.py')]
files_in_dir = []

for file_path in files_to_check:
  try:
    file_type = get_file_type(file_path)
    if file_type in SUPPORTED_FILE_TYPES:
        files_in_dir.append(file_path)
  except NotImplemented:
    continue

In [ ]:
# add the documents to the project

for doc_file in files_in_dir:
  Document(file_path=doc_file, project=my_project).save()

Note:  
If you prefer, use the example documents in the [example folder](https://drive.google.com/drive/folders/1pjJkfIDh07bvbOp4-YOM32aOC4iGUjwW?usp=sharing).

The documents are part of the public dataset [rvl-cdip](https://www.cs.cmu.edu/~aharley/rvl-cdip/).

Alternatively, you can do this step directly on the app by uploading the documents there.  
Follow the instructions in the readme of the [repository](https://github.com/konfuzio-ai/Python-SDK).

Take a look into the text of a document to check its quality.

In [ ]:
# get the documents from the project
my_project.update()
documents = my_project.get_documents_from_project()

doc = documents[0]
print(doc.text)

Use a model to create some first annotations.

Here we will use a pre trained model for Named-entity Recognition (NER) from the flair library. You can use your **own model**.

In [ ]:
model = SequenceTagger.load('ner-ontonotes-fast')

Check which labels are in the flair model and chose some.

In [ ]:
labels_available = list(set([tag.split('-')[1] for tag in model.tag_dictionary.get_items() if '-' in tag]))
print(labels_available)

Here we will use the pretrained model to have some first annotations for **dates**, **organizations** and **persons** in the documents.

In [ ]:
# labels for which we want to create annotations
labels_names = ['DATE', 'ORG', 'PERSON']

We need to create the labels in the project accordingly with the labels_names defined above.

In [ ]:
for label in labels_names:
    Label(project=my_project, has_multiple_top_candidates=True, text=label).save()

Collect the defined labels and the correspondent template.

In [ ]:
# labels defined in the Konfuzio project that match the ones we want
my_labels = [label for label in my_project.labels if label.name in labels_names]

# correspondent template id
my_templates_ids = [my_label.templates[0].id for my_label in my_labels]

Post the results from the pretrained model as annotations in the Konfuzio app.

In [ ]:
def annotate_document(document, annotations, labels, template_ids):
    """
    Create annotations in a document.

    Based on a list of possible annotations, filter the ones that match with the
    labels in the project.

    :param document: document object
    :param annotations: list of possible annotations
    :param labels: labels for which the annotations can be created
    :param template_ids: templates ids that correspondent to the labels
    """
    labels_names = [label.name for label in labels]

    for entity in annotations:
        if entity.tag in labels_names:
            for label, template_id in zip(labels, template_ids):
                if label.name == entity.tag:
                    break
            annotation_obj = Annotation(
                document=document,
                document_id=document.id,
                start_offset=entity.start_pos,
                end_offset=entity.end_pos,
                label=label.id,
                template_id=template_id,
                accuracy=None
            )
            _ = annotation_obj.save()

In [ ]:
for doc in documents:
    s = Sentence(doc.text)
    model.predict(s)
    first_results = s.get_spans('ner')
    annotate_document(doc, first_results, my_labels, my_templates_ids)

You can view the results in the SmartView and **revise** them by accepting or rejecting as described in the point  [7. Give feedback](https://konfuzio.com/en/data-extraction-documents-how-to-1/#how-to-step-1611219481233) of "How to start your project with Konfuzio".

Now you can use the revised annotations to **train a model in the app** or **retrain your local model**.

If you want to train a model in the app, you have first to specify which documents should be used for training.
You can do that in the app by selecting "Add to training data set" as described in the point  [5. Division into training and test data](https://konfuzio.com/en/data-extraction-documents-how-to-1/#how-to-step-1611219345671).

Then, you just have to set the retraining of the project by ticking the box and select the respective execution in the action field [here](https://app.konfuzio.com/admin/server/project/). More detailed instructions in the point [6.Start retraining and evaluate results](https://konfuzio.com/en/data-extraction-documents-how-to-1/#how-to-step-1611219370716).

If you want to retrain your local model, you have to update your local project first so that you get the information of the revised annotations.

In [ ]:
my_project.update()

The next step is to define which documents should be used for training.

In [ ]:
# define all uploaded documents as training dataset
for doc in documents:
    doc.dataset_status = 2
    doc.save()

The training documents can now be accessed by the attribute **documents** of the project.

In [ ]:
print(len(my_project.documents())

Then, use the reviewed annotations to retrain your model.

To get the annotations in a document, you just have to use the method **annotations()**.

The cell bellow shows how many annotations are in the first document in the training set.

In [ ]:
print(len(my_project.documents[0].annotations()))

An example of how to get the data in the format necessary to retrain the flair model is shown bellow.

This example is merely demonstrative. You might want to adapt the data structure and training parameters to your case and/or model.

In this case, we only need to get the **start** and **end offsets** of each annotation and its **label**.

The flair model requires the data structure to be in the BIO scheme and to be saved in a text file.

In [ ]:
def convert_to_bio_scheme(text, annotations):
    """
    Mark all the entities in the text as per the BIO scheme.

    :param text: text of the document
    :param annotations: annotations in the document with star and end offset and label name
    :return: dictionary with the label of each word in the text
    """
    word_dict = {}
    for word in text.split():
        word_dict[word] = 'O'

    for start, end, label_name in annotations:
        temp_str = text[start:end]
        tmp_list = temp_str.split()
        if len(tmp_list) > 1:
            word_dict[tmp_list[0]] = 'B-' + label_name
            for w in tmp_list[1:]:
                word_dict[w] = 'I-' + label_name
        else:
            word_dict[temp_str] = 'B-' + label_name
    return word_dict

In [ ]:
def create_data(file_name, documents):
    """
    Create data structure in the format expected by flair and saves it in a txt file.

    :param file_name: name of the file where to save the data
    :param documents: dataset documents
    """
    with open(file_name, 'w') as f:
        for doc in documents:
            document_annotations = doc.annotations()

            if len(document_annotations) > 0:
                annotations_in_doc = []

                for annotation in document_annotations:
                    data = (annotation.start_offset, annotation.end_offset, annotation.label.name)
                    annotations_in_doc.append(data)

                converted_annots = convert_to_bio_scheme(doc.text, annotations_in_doc)

                for i in converted_annots.keys():
                    f.writelines(i + ' ' + converted_annots[i] + '\n')
                f.writelines('\n')


In [ ]:
n_val = max(round(len(my_project.documents) * 0.3), 1)

create_data('train.txt', my_project.documents[:-n_val])
create_data('dev.txt', my_project.documents[-n_val:])
create_data('test.txt', my_project.test_documents)

After having the data in the expected structure, we can retrain the tagger model loaded in the begining.

In [ ]:
def retrain(model, data_folder='./'):
    """
    Train the existing model for a few more epochs.

    :param tagger: pre trained model
    :param data_folder: folder with the dataset files.
    """
    # define columns
    columns = {0: 'text', 1: 'ner'}

    # initializing the corpus
    with open('train.txt') as train_file:
      train_file.seek(0)
      if not train_file.read(1):
        return "There is no training data. Please assign a training data."
  
    corpus = ColumnCorpus(data_folder, columns, train_file='train.txt', test_file='test.txt', dev_file='dev.txt')

    model = ModelTrainer(model, corpus)
    model.train('resources/taggers/continued_model',
                learning_rate=0.01,
                mini_batch_size=32,
                max_epochs=10)

In [ ]:
# retrain the model
retrain(model)

Repeat the process if you would like.

Thank you!